# Домашнее задание №5.
# Part-of-Speech разметка, NER, извлечение отношений

## <a id='Оглавление_5'>Оглавление</a>

## [Подготовка ](#Подготовка_5)

## [Задание 1](#Задание_1)
* [1.1 Теггеры UnigramTagger, BigramTagger, TrigramTagger](#Теггеры)
* [1.2 Самописный теггер](#Самописный_теггер)
* [1.3 Выводы](#Выводы_5.1)

## [Задание 2](#Задание_2)
* [2.1 NER из NLTK](#NER_NLTK)
* [2.2 NER из Spacy](#NER_Spacy)
* [2.3 NER из slovnet](#NER_slovnet)
* [2.4 NER из deeppavlov](#NER_deeppavlov)
* [2.5 Самописный NER](#NER_свой)
* [2.6 Выводы](#Выводы_5.2)

## <a id='Подготовка_5'>Подготовка</a>

In [1]:
%pip install pyconll
%pip install corus
%pip install -U spacy
%pip install slovnet
%pip install deeppavlov
%pip install ipymarkup
%pip install sklearn_crfsuite

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 441 kB 1.7 MB/s            
     |████████████████████████████████| 54 kB 3.9 MB/s             
     |████████████████████████████████| 42 kB 995 kB/s             
  

In [1]:
#!:python3
%%python3 -m spacy download ru_core_news_sm
%%python3 -m deeppavlov install squad_bert
%%python3 -m deeppavlov install ner_ontonotes

Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


2023-01-29 20:14:09.411512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64
2023-01-29 20:14:09.411543: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-29 20:14:09.411561: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (s-410d2fa1-7f99-4bf1-85f8-a124ef9b415e): /proc/driver/nvidia/version does not exist
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [31]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pyconll
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
import os
import corus
import deeppavlov
from deeppavlov import configs, build_model

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
import spacy
from spacy import displacy
import ru_core_news_sm
import pandas as pd
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

## <a id='Задание_1'>Задание 1. Написать теггер на данных с русским языком</a>

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации;
2. написать свой теггер как на занятии (попробовать разные векторайзеры, добавить знание не только букв, но и слов);
3. сравнить все реализованные методы сделать выводы.

In [4]:
# Загружаем данные
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
# Просматриваем 2 предложения тренировочных данных:
# В каждом из них для каждого токена выводим form (словоформа для чтения) и upos (часть речи)
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [6]:
# Собираем данные
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [7]:
# Определяем наибольшую длину предложения и токена
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [8]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [9]:
# Просматриваем тренировочные данные (токен, тег)
fdata_train[:5]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

### <a id='Теггеры'>1.1 Теггеры UnigramTagger, BigramTagger, TrigramTagger и их комбмнации</a>

**UnigramTagger** ( UnigramTagger учитывает условную частоту тегов и предсказывает наиболее частый тег для каждого токена, не ориентируется на соседние слова)

In [10]:
unigram_tagger = UnigramTagger(fdata_train)
accuracy_U = unigram_tagger.evaluate(fdata_test)
display(accuracy_U)

0.823732013802982

**BigramTagger** (BigramTagger учитывает тэги двух слов: текущее и предыдущее слово)

In [11]:
bigram_tagger = BigramTagger(fdata_train)
accuracy_B = bigram_tagger.evaluate(fdata_test)
display(accuracy_B)

0.6093886320724006

**TrigramTagger** (TrigramTagger учитывает тэги трёх слов: текущее и 2 предыдущих слова)

In [12]:
trigram_tagger = TrigramTagger(fdata_train)
accuracy_T = trigram_tagger.evaluate(fdata_test)
display(accuracy_T)

0.1778631421316492

**Комбинация теггеров**

In [13]:
list_1 = [UnigramTagger, BigramTagger]

list_2 = [UnigramTagger, TrigramTagger]

list_3 = [BigramTagger, TrigramTagger]

list_4 = [UnigramTagger, BigramTagger, TrigramTagger]

accuracy_N = []

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

for list_N in [list_1, list_2, list_3, list_4]:
    backoff = DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         list_N,  
                         backoff = backoff) 

    accuracy_N.append(tag.evaluate(fdata_test))

accuracy_N

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [14]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 'UnigramTagger+BigramTagger+TrigramTagger'], 'Accuracy' : [accuracy_U, accuracy_B, accuracy_T] + accuracy_N})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


**Вывод:** лучшие показатели метрики у комбинации теггеров UnigramTagger + BigramTagger + TrigramTagger

### <a id='Самописный_теггер'>1.2 Самописный теггер с различными векторайзерами</a>

In [15]:
# Собираем токены и лэйблы в списки
train_tok = [] # Список токенов
train_label = [] # Список теггов
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [16]:
# Кодируем целевые метки значениями от 0 до (кол-во классов - 1)
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [17]:
# Строим модели логистической регрессии с различными векторайзерами
# P.S. тестовые данные срезаны до 115000 штук из-за битых файлов

vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [18]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927826
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921185
9,"HashingVectorizer(analyzer='char', n_features=...",0.903654
8,"HashingVectorizer(analyzer='char', n_features=...",0.901222
7,"HashingVectorizer(analyzer='char', n_features=...",0.897174
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882215
3,"CountVectorizer(ngram_range=(1, 3))",0.672425
4,"TfidfVectorizer(ngram_range=(1, 3))",0.662678
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.620668


In [19]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929504
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923609
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905000
7,"HashingVectorizer(analyzer='char', n_features=...",0.901417
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885157
3,"CountVectorizer(ngram_range=(1, 3))",0.664722
4,"TfidfVectorizer(ngram_range=(1, 3))",0.652991
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.630270


**Вывод:** лучшие показатели метрик f1 и accuracy у CountVectorizer(analyzer='char', ngram_range=(1, 3))

### <a id='Выводы_5.1'>1.3 Выводы</a>

In [20]:
result.sort_values('Accuracy', ascending=False).head(1)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905


In [27]:
result_model.sort_values('f1_score', ascending=False).head(1)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927826


In [28]:
result_model_acc.sort_values('Accuracy', ascending=False).head(1)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929504


***Вывод: лучший метод pos tagging (по результатам подсчета метрик) - самописный теггер с CountVectorizer(analyzer='char', ngram_range=(1, 3))***

## <a id='Задание_2'>Задание 2. Проверить насколько хорошо работает NER</a>

Данные брать из http://www.labinform.ru/pub/named_entities/

1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы:
  * передаём в сетку токен и его соседей
  * передаём в сетку только токен
  * свой вариант
3. сравнить реализованные подходы на качество (вывести precision/recall/f1_score)

Датасет содержит 1000 файлов txt и 1000 файлов ann, в которых размечены 3 сущности:

- имена людей (PER),
- имена организаций (ORG),
- географические названия (LOC).

Файлы 595.txt и 595.ann битые, в следcтвие чего были удалены из рассмотрения.

In [21]:
# Просматриваем содержание коллекции
print(os.listdir("collection3"))

['03_12_12a.ann', '003.txt', '03_12_12c.ann', '03_12_12d.txt', '03_12_12c.txt', '03_12_12h.ann', '03_12_12h.txt', '004.ann', '03_12_12g.txt', '03_12_12b.ann', '03_12_12a.txt', '03_12_12g.ann', '003.ann', '03_12_12d.ann', '03_12_12b.txt', '004.txt', '04_03_13a_sorokin.ann', '04_02_13a_abdulatipov.ann', '04_12_12f.txt', '04_02_13a_abdulatipov.txt', '04_12_12f.ann', '04_12_12g.txt', '04_12_12h_corr.ann', '04_12_12b.txt', '04_12_12b.ann', '04_12_12d.ann', '006.txt', '007.ann', '04_12_12g.ann', '005.txt', '009.txt', '04_12_12h_corr.txt', '005.ann', '008.txt', '006.ann', '09_01_13.ann', '008.ann', '007.txt', '009.ann', '09_01_13a.txt', '09_01_13.txt', '09_01_13a.ann', '04_03_13a_sorokin.txt', '09_01_13c.txt', '09_01_13e.ann', '09_01_13c.ann', '09_01_13e.txt', '09_01_13d.txt', '09_01_13h.txt', '010.ann', '09_01_13i.ann', '09_01_13d.ann', '09_01_13h.ann', '010.txt', '09_01_13i.txt', '11_01_13b.txt', '10_01_13a.ann', '11_01_13e.txt', '10_01_13i.ann', '10_01_13d.ann', '04_12_12d.txt', '11_01_13e

In [22]:
# Собираем только текстовые файлы коллекции
fileDir = r"collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_txt)

['003.txt', '03_12_12d.txt', '03_12_12c.txt', '03_12_12h.txt', '03_12_12g.txt', '03_12_12a.txt', '03_12_12b.txt', '004.txt', '04_12_12f.txt', '04_02_13a_abdulatipov.txt', '04_12_12g.txt', '04_12_12b.txt', '006.txt', '005.txt', '009.txt', '04_12_12h_corr.txt', '008.txt', '007.txt', '09_01_13a.txt', '09_01_13.txt', '04_03_13a_sorokin.txt', '09_01_13c.txt', '09_01_13e.txt', '09_01_13d.txt', '09_01_13h.txt', '010.txt', '09_01_13i.txt', '11_01_13b.txt', '11_01_13e.txt', '04_12_12d.txt', '10_01_13d.txt', '10_01_13i.txt', '013.txt', '10_01_13a.txt', '014.txt', '011.txt', '14_01_13g.txt', '012.txt', '15_01_13a.txt', '14_01_13c.txt', '015 (!).txt', '14_01_13i.txt', '15_01_13b.txt', '15_01_13e.txt', '017.txt', '016.txt', '15_01_13f.txt', '019.txt', '018.txt', '19_11_12d.txt', '020.txt', '19_11_12h.txt', '20_11_12a.txt', '20_11_12d.txt', '20_11_12b.txt', '20_11_12c.txt', '20_11_12i.txt', '021.txt', '21_11_12c.txt', '21_11_12h.txt', '21_11_12i.txt', '21_11_12j.txt', '022.txt', '22_11_12a.txt', '22

In [23]:
# Заносим данные файлов txt в датасет
text_list = []
for file in documents_txt:
    doc = open('collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text
0,"Пулеметы, автоматы и снайперские винтовки изъя..."
1,Глава News International Т.Мокридж покидает св...
2,"\nОпрос РБК-Украина: Читатели уверены, что Аза..."
3,Названы кандидаты на пост украинского премьера...
4,Из-за подчиненного-взяточника должностей лишил...
...,...
692,"Эксперт пояснил, как текст про ""отставку"" Якун..."
693,Сванидзе не одобрил идею о выдвижении Сноудена...
694,Путин: Позиция Ирана на международной арене за...
695,Чавес назначил нового вице-президента\nПрезиде...


### <a id='NER_NLTK'>2.1 NER из NLTK</a>

In [24]:
# Требуется для токенизации
nltk.download('punkt')
# Требуется для parts of speech tagging
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [25]:
# Пример текста
document = data_text.text[0]

# Разбиваем документ на токены и применяем pos tagging (на выходе список кортежей (токен, часть речи))
nltk.pos_tag(nltk.word_tokenize(document))

[('Пулеметы', 'NN'),
 (',', ','),
 ('автоматы', 'NNP'),
 ('и', 'NNP'),
 ('снайперские', 'NNP'),
 ('винтовки', 'NNP'),
 ('изъяты', 'NNP'),
 ('в', 'NNP'),
 ('арендуемом', 'NNP'),
 ('американцами', 'NNP'),
 ('доме', 'NNP'),
 ('в', 'NNP'),
 ('Бишкеке', 'VBD'),
 ('05/08/2008', 'CD'),
 ('10:35', 'CD'),
 ('БИШКЕК', 'NN'),
 (',', ','),
 ('5', 'CD'),
 ('августа', 'JJ'),
 ('/Новости-Грузия/', 'JJ'),
 ('.', '.'),
 ('Правоохранительные', 'JJ'),
 ('органы', 'JJ'),
 ('Киргизии', 'NN'),
 ('обнаружили', 'NNP'),
 ('в', 'NNP'),
 ('доме', 'NNP'),
 (',', ','),
 ('арендуемом', 'NNP'),
 ('гражданами', 'NNP'),
 ('США', 'NNP'),
 ('в', 'NNP'),
 ('Бишкеке', 'NNP'),
 (',', ','),
 ('пулеметы', 'NNP'),
 (',', ','),
 ('автоматы', 'NNP'),
 ('и', 'NNP'),
 ('снайперские', 'NNP'),
 ('винтовки', 'NNP'),
 (',', ','),
 ('сообщает', 'NNP'),
 ('во', 'NNP'),
 ('вторник', 'NNP'),
 ('пресс-служба', 'JJ'),
 ('МВД', 'NNP'),
 ('Киргизии', 'NNP'),
 ('.', '.'),
 ('``', '``'),
 ('В', 'JJ'),
 ('ходе', 'JJ'),
 ('проведения', 'JJ'),
 (

In [32]:
# Распознаем именнованные сущности с помощью классификатора (Person, Organization, GPE)
{(' '.join(c[0] for c in chunk), chunk.label()) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label')}

{('БИШКЕК', 'ORGANIZATION'),
 ('Киргизии', 'ORGANIZATION'),
 ('Киргизии', 'PERSON'),
 ('МВД', 'ORGANIZATION'),
 ('МВД Киргизии', 'ORGANIZATION'),
 ('Оружие', 'PERSON'),
 ('Пулеметы', 'GPE'),
 ('РИА Новости', 'ORGANIZATION'),
 ('США', 'ORGANIZATION'),
 ('Сейчас', 'PERSON')}

In [46]:
# Разметка из collection3
pd.read_csv('collection3/003.ann', delimiter='\t')

,T1,LOC 82 89,Бишкеке
0,T2,LOC 113 119,БИШКЕК
1,T3,ORG 132 146,Новости-Грузия
2,T4,LOC 175 183,Киргизии
3,T5,LOC 225 228,США
4,T6,LOC 231 238,Бишкеке
5,T7,ORG 316 319,МВД
6,T8,LOC 320 328,Киргизии
7,T9,LOC 492 500,Киргизии
8,T10,LOC 525 528,США
9,T11,ORG 955 958,МВД


**Вывод:** NER из nltk нашла все сущности.

### <a id='NER_Spacy'>2.2 NER из Spacy</a>

In [34]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [35]:
# Список токенов, частей речи и сущностей
for token in article:
    print(token.text, token.pos_, token.dep_)

Пулеметы NOUN nsubj:pass
, PUNCT punct
автоматы NOUN conj
и CCONJ cc
снайперские ADJ amod
винтовки NOUN conj
изъяты VERB ROOT
в ADP case
арендуемом ADJ amod
американцами NOUN amod
доме NOUN obl
в ADP case
Бишкеке PROPN nmod


 SPACE dep
05/08/2008 NUM appos
10:35 NUM appos


 SPACE dep
БИШКЕК PROPN obl
, PUNCT punct
5 ADJ obl
августа NOUN flat
/Новости NOUN obj
- NOUN obj
Грузия/. PROPN obj
Правоохранительные ADJ amod
органы NOUN nsubj
Киргизии PROPN nmod
обнаружили VERB conj
в ADP case
доме NOUN obl
, PUNCT punct
арендуемом VERB acl
гражданами NOUN obl:agent
США PROPN nmod
в ADP case
Бишкеке PROPN nmod
, PUNCT punct
пулеметы NOUN conj
, PUNCT punct
автоматы NOUN conj
и CCONJ cc
снайперские ADJ amod
винтовки NOUN conj
, PUNCT punct
сообщает VERB parataxis
во ADP case
вторник NOUN obl
пресс NOUN nsubj
- NOUN nsubj
служба NOUN nsubj
МВД PROPN nmod
Киргизии PROPN nmod
. PUNCT punct


 SPACE dep
" PUNCT punct
В ADP case
ходе NOUN fixed
проведения NOUN obl
оперативно ADJ amod
- ADJ amod
про

In [47]:
# Разметка из collection3
pd.read_csv('collection3/003.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 82 89,Бишкеке
1,T2,LOC 113 119,БИШКЕК
2,T3,ORG 132 146,Новости-Грузия
3,T4,LOC 175 183,Киргизии
4,T5,LOC 225 228,США
5,T6,LOC 231 238,Бишкеке
6,T7,ORG 316 319,МВД
7,T8,LOC 320 328,Киргизии
8,T9,LOC 492 500,Киргизии
9,T10,LOC 525 528,США


**Вывод:** NER из spacy нашла все сущности.

### <a id='NER_slovnet'>2.3 NER из slovnet</a>

In [48]:
text = data_text.text[0]
navec = Navec.load('slovnet/navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet/slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

FileNotFoundError: [Errno 2] No such file or directory: 'slovnet/navec_news_v1_1B_250K_300d_100q.tar'

**Вывод:** NER из slovnet 

### <a id='NER_deeppavlov'>2.4 NER из deeppavlov</a>

In [49]:
deeppavlov_ner = build_model(configs.ner, download=True)
rus_document = data_text.text[0]
deeppavlov_ner([rus_document])

AttributeError: 'Struct' object has no attribute 'get'

### <a id='NER_свой'>2.5 Самописный NER</a>

In [50]:
# Собираем только ann файлы коллекции
fileDir = r"collection3"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['03_12_12a.ann', '03_12_12c.ann', '03_12_12h.ann', '004.ann', '03_12_12b.ann', '03_12_12g.ann', '003.ann', '03_12_12d.ann', '04_03_13a_sorokin.ann', '04_02_13a_abdulatipov.ann', '04_12_12f.ann', '04_12_12h_corr.ann', '04_12_12b.ann', '04_12_12d.ann', '007.ann', '04_12_12g.ann', '005.ann', '006.ann', '09_01_13.ann', '008.ann', '009.ann', '09_01_13a.ann', '09_01_13e.ann', '09_01_13c.ann', '010.ann', '09_01_13i.ann', '09_01_13d.ann', '09_01_13h.ann', '10_01_13a.ann', '10_01_13i.ann', '10_01_13d.ann', '11_01_13e.ann', '011.ann', '11_01_13b.ann', '014.ann', '14_01_13i.ann', '14_01_13c.ann', '15_01_13a.ann', '14_01_13g.ann', '013.ann', '15_01_13e.ann', '012.ann', '016.ann', '018.ann', '019.ann', '017.ann', '15_01_13b.ann', '19_11_12d.ann', '15_01_13f.ann', '015 (!).ann', '19_11_12h.ann', '20_11_12a.ann', '020.ann', '20_11_12b.ann', '20_11_12d.ann', '20_11_12c.ann', '20_11_12i.ann', '021.ann', '21_11_12c.ann', '21_11_12h.ann', '21_11_12i.ann', '21_11_12j.ann', '022.ann', '22_11_12a.ann', '22

In [55]:
ann = pd.read_csv('collection3/003.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 82 89,Бишкеке
1,T2,LOC 113 119,БИШКЕК
2,T3,ORG 132 146,Новости-Грузия
3,T4,LOC 175 183,Киргизии
4,T5,LOC 225 228,США
5,T6,LOC 231 238,Бишкеке
6,T7,ORG 316 319,МВД
7,T8,LOC 320 328,Киргизии
8,T9,LOC 492 500,Киргизии
9,T10,LOC 525 528,США


In [59]:
# Составляем списки токенов и интенсов (из файла .ann делается словарь {слово : интенс}, из словаря каждому токену сопоствляем интенс)
docs = []
for i in range(len(documents_ann)):
    words = []
    labels = []
    # Подготавливаем текст
    text = data_text['text'][i]
    
    df = pd.read_csv('collection3/' + documents_ann[i], delimiter='\t', header=None)
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [60]:
print(docs[0][0]), print(docs[0][1])

['Пулеметы', ',', 'автоматы', 'и', 'снайперские', 'винтовки', 'изъяты', 'в', 'арендуемом', 'американцами', 'доме', 'в', 'Бишкеке', '05/08/2008', '10', ':', '35', 'БИШКЕК', ',', '5', 'августа', '/', 'Новости-Грузия', '/', '.', 'Правоохранительные', 'органы', 'Киргизии', 'обнаружили', 'в', 'доме', ',', 'арендуемом', 'гражданами', 'США', 'в', 'Бишкеке', ',', 'пулеметы', ',', 'автоматы', 'и', 'снайперские', 'винтовки', ',', 'сообщает', 'во', 'вторник', 'пресс-служба', 'МВД', 'Киргизии', '.', '"', 'В', 'ходе', 'проведения', 'оперативно-профилактического', 'мероприятия', 'под', 'кодовым', 'названием', '"', 'Арсенал', '"', 'в', 'новостройке', 'Ынтымак', ',', 'в', 'доме', ',', 'принадлежащем', '66-летнему', 'гражданину', 'Киргизии', 'и', 'арендуемом', 'гражданами', 'США', ',', 'обнаружены', 'и', 'изъяты', ':', 'шесть', 'крупнокалиберных', 'пулеметов', 'с', 'оптическим', 'прицелом', 'и', 'с', 'приборами', 'ночного', 'видения', ',', '26', 'автоматов', 'калибра', '5,56', 'миллиметра', ',', 'два',

(None, None)

In [61]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Пулеметы	OUT
,	OUT
автоматы	OUT
и	OUT
снайперские	OUT
винтовки	OUT
изъяты	OUT
в	OUT
арендуемом	OUT
американцами	OUT
доме	OUT
в	OUT
Бишкеке	OUT
05/08/2008	OUT
10	OUT
:	OUT
35	OUT
БИШКЕК	OUT
,	OUT
5	OUT
августа	OUT
/	OUT
Новости-Грузия	OUT
/	OUT
.	OUT
Правоохранительные	OUT
органы	OUT
Киргизии	OUT
обнаружили	OUT
в	OUT
доме	OUT
,	OUT
арендуемом	OUT
гражданами	OUT
США	OUT
в	OUT
Бишкеке	OUT
,	OUT
пулеметы	OUT
,	OUT
автоматы	OUT
и	OUT
снайперские	OUT
винтовки	OUT
,	OUT
сообщает	OUT
во	OUT
вторник	OUT
пресс-служба	OUT
МВД	OUT
Киргизии	OUT
.	OUT
"	OUT
В	OUT
ходе	OUT
проведения	OUT
оперативно-профилактического	OUT
мероприятия	OUT
под	OUT
кодовым	OUT
названием	OUT
"	OUT
Арсенал	OUT
"	OUT
в	OUT
новостройке	OUT
Ынтымак	OUT
,	OUT
в	OUT
доме	OUT
,	OUT
принадлежащем	OUT
66-летнему	OUT
гражданину	OUT
Киргизии	OUT
и	OUT
арендуемом	OUT
гражданами	OUT
США	OUT
,	OUT
обнаружены	OUT
и	OUT
изъяты	OUT
:	OUT
шесть	OUT
крупнокалиберных	OUT
пулеметов	OUT
с	OUT
оптическим	OUT
прицелом	OUT
и	OUT
с	OUT
приборами	OU

In [62]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Пулеметы,OUT
1,0,",",OUT
2,0,автоматы,OUT
3,0,и,OUT
4,0,снайперские,OUT
5,0,винтовки,OUT
6,0,изъяты,OUT
7,0,в,OUT
8,0,арендуемом,OUT
9,0,американцами,OUT


**Будем передавать в модель токен с соседями**

In [63]:
# Преобразуем датасет: 
# Группируем записи по номеру предложения (аггфункция - список кортежей (токен, лейбл))
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

# Преобразует датасет df
getter = SentenceGetter(df)
# Получаем список номеров преложений датасета df
sentences = getter.sentences

In [64]:
# Преобразуем слова в признаки:
# Для каждого слова-токена составляем словарь признаков, 
# Определяем является ли оно начаотным, конечным в предложении
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

# Преобразуем предложения:
# Каждое слово предложения преобразуем в признаки
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

# Список всех лейблов предложения
def sent2labels(sent):
    return [label for token, label in sent]

# Список всех токенов предложения
def sent2tokens(sent):
    return [token for token, label in sent]

In [65]:
# Собираем данные и метки
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [66]:
# Передаем в датасет токен с соседями (первый индекс -- предложение, 2 индекс -- слово)
X[2][7]

{'bias': 1.0,
 'word.lower()': 'азаров',
 'word[-3:]': 'ров',
 'word[-2:]': 'ов',
 'word.isdigit()': False,
 '-1:word.lower()': 'что',
 '+1:word.lower()': 'останется'}

In [67]:
len(X)

696

In [68]:
# Разбиваем данные на train и test
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [69]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Градиентный спуск с использованием метода L-BFGS
    c1=0.1, #Коэффициент для регуляризации L1
    c2=0.1, #Коэффициент для регуляризации L2
    max_iterations=1000, #Максимальное количество итераций
    all_possible_transitions=True, #Генерация объектов (не встречающихся в обучающих данных)
    verbose=True #Включение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 696/696 [00:01<00:00, 684.07it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 88941
Seconds required: 0.224

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.11  loss=89535.40 active=88196 feature_norm=1.00
Iter 2   time=0.06  loss=67149.76 active=84311 feature_norm=1.80
Iter 3   time=0.06  loss=66018.52 active=87657 feature_norm=1.70
Iter 4   time=0.06  loss=65185.83 active=86205 feature_norm=1.58
Iter 5   time=0.06  loss=64640.91 active=88260 feature_norm=1.60
Iter 6   time=0.06  loss=55069.05 active=77481 feature_norm=3.72
Iter 7   time=0.12  loss=53204.03 active=78685 feature_norm=4.71
Iter 8   time=0.06  loss=50874.16 active=79450 feature_norm=5.30
Iter 9   time=0.06  loss=47660.17 active=78241 feature_norm=7.02
Iter 10  time

In [70]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [71]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.0

### <a id='Выводы_5.2'>2.6 Выводы</a>

Готовые решения NER из spacy и nltk показывают хорошие результаты в задачах извлечения именованных сущностей. Не получилось извлечь NER с помощью готовых решений из slovnet и deeppavlov. При составлении собственного алгоритма NER не получилось определить метрики.